In [46]:
from sklearn.linear_model import LogisticRegressionCV
import pandas as pd
import numpy as np

In [47]:
for data in ["antenatal"]:    
    for outcome in ['_hie']:
        
        print("Running LR on {} for {}".format(data, outcome))
        
        # read in data
        train = pd.read_csv("data/{}{}_train.csv".format(data, outcome), index_col=0).astype('float32')
        test = pd.read_csv("data/{}{}_test.csv".format(data, outcome), index_col=0).astype('float32')
        
        # downsample for testing
        train = train.sample(n=1000, random_state=1)
           
        # split outcome from predictors
        train_y = train.pop(outcome)
        test_y = test.pop(outcome)
        
        # evaluate model
        clf = LogisticRegressionCV(cv=5, random_state=0, penalty="l1", solver="saga", n_jobs=-1, max_iter=1000000, scoring="roc_auc")
        clf.fit(train, train_y)

Running LR on antenatal for _hie


In [50]:
clf.score(train, train_y)


0.6759410801963994

In [59]:
clf.scores_

{1.0: array([[0.27777778, 0.72222222, 0.84090909, 0.88131313, 0.88636364,
         0.88888889, 0.88888889, 0.88888889, 0.88888889, 0.88888889],
        [0.50252525, 0.47474747, 0.55555556, 0.61111111, 0.5959596 ,
         0.59343434, 0.5959596 , 0.5959596 , 0.5959596 , 0.5959596 ],
        [0.68697124, 0.59390863, 0.39763113, 0.2927242 , 0.25549915,
         0.2605753 , 0.2605753 , 0.2605753 , 0.2605753 , 0.2605753 ],
        [0.4500846 , 0.48900169, 0.63790186, 0.65313029, 0.66159052,
         0.66328257, 0.66328257, 0.66328257, 0.66328257, 0.66328257],
        [0.40186125, 0.53130288, 0.62774958, 0.59729272, 0.58544839,
         0.58544839, 0.58544839, 0.58544839, 0.58544839, 0.58544839]])}

In [57]:
train

,al_fage_0374,al_fertilitylength_0071,al_gestatregistration_0034,al_gravidity_0044,al_mage_0031,al_magecat_0033,al_mdbp_0334,al_medu_0270,al_menarche_0312,al_mhct_0089,...,ac_viral_0131_0.0,ac_viral_0131_2.0,ac_viral_0245_0.0,ac_viral_0245_1.0,ac_viral_0245_2.0,ac_viral_0245_3.0,ac_viral_0245_4.0,ac_viral_0245_5.0,ac_viral_0245_6.0,ac_viral_0245_7.0
_id,,,,,,,,,,,,,,,,,,,,,
5102451,21.0,0.0,14.0,0.0,18.0,2.0,64.0,12.0,13.0,380.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66132651,48.0,0.0,24.0,11.0,39.0,6.0,80.0,7.0,14.0,280.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50113931,26.0,0.0,19.0,1.0,21.0,3.0,70.0,14.0,12.0,390.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66139031,26.0,0.0,19.0,0.0,26.0,4.0,70.0,12.0,15.0,300.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66124091,29.0,0.0,15.0,1.0,28.0,4.0,90.0,11.0,12.0,320.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5101701,36.0,0.0,31.0,7.0,35.0,6.0,66.0,12.0,17.0,340.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5132761,25.0,2.0,16.0,0.0,22.0,3.0,70.0,11.0,11.0,370.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82108871,24.0,0.0,35.0,2.0,21.0,3.0,60.0,7.0,14.0,290.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
